In [2]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm 
import glob
import re
import requests
import io

In [7]:
pfamA_motors = pd.read_csv("../../data/pfamA_motors.csv")
pfamA_motors.head()

,Unnamed: 0,id,description,seq,pfamA_acc,clan
0,0,A0A495CYV6_9MYCO/3-388,A0A495CYV6_9MYCO/3-388 A0A495CYV6.1 PF00871.18...,AVLVVNSGSSSIKYQVIDEQSGDRLAQGLVERIGESGRGRVVYKGA...,PF00871,actin_like
1,1,A0A3A6QL58_9VIBR/4-390,A0A3A6QL58_9VIBR/4-390 A0A3A6QL58.1 PF00871.18...,LVLVLNCGSSSLKFAIVDAETGAEHLTGLAECLGLPEARMKWKLDG...,PF00871,actin_like
2,2,A0A2T0AKP1_9THEO/2-389,A0A2T0AKP1_9THEO/2-389 A0A2T0AKP1.1 PF00871.18...,KILVLNCGSSSVKYQLFDMQREEVMARGLVERIGITGSMLTHRPAG...,PF00871,actin_like
3,3,H1XW95_9BACT/146-327,H1XW95_9BACT/146-327 H1XW95.1 PF00871.18;Aceta...,ISGMPLIPRKSIFHALNQKAVARETAKKLGKKYRESSIIVAHMGGG...,PF00871,actin_like
4,4,A0A396TZH3_9GAMM/13-397,A0A396TZH3_9GAMM/13-397 A0A396TZH3.1 PF00871.1...,AILVINCGSSSVKFSLIHPKTGQTILSGLAECLLANDAVIKIKFDN...,PF00871,actin_like


In [9]:
kinesin = pfamA_motors.loc[pfamA_motors["pfamA_acc"] == "PF00225",:] 
kinesin.shape

(15099, 6)

In [10]:
tubulin = pfamA_motors.loc[pfamA_motors["pfamA_acc"] == "PF00091",:] 
tubulin.shape

(5951, 6)

In [12]:
kinesin["uniprot_acc"] = kinesin["id"].apply(lambda s: s.split("/")[0])
tubulin["uniprot_acc"] = tubulin["id"].apply(lambda s: s.split("/")[0])

<ipython-input-12-c8d412e88353>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kinesin["uniprot_acc"] = kinesin["id"].apply(lambda s: s.split("/")[0])
<ipython-input-12-c8d412e88353>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tubulin["uniprot_acc"] = tubulin["id"].apply(lambda s: s.split("/")[0])


In [15]:
kinesin.head()

,Unnamed: 0,id,description,seq,pfamA_acc,clan,uniprot_acc
1407315,1407315,V2XA84_MONRO/12-342,V2XA84_MONRO/12-342 V2XA84.1 PF00225.24;Kinesin;,RLRPRIDGEIDDGAIQVCHASDNTAASSSSASSTTSLSYISAPNPR...,PF00225,p_loop_gtpase,V2XA84_MONRO
1407316,1407316,A0A0D2I9H7_9EURO/73-404,A0A0D2I9H7_9EURO/73-404 A0A0D2I9H7.1 PF00225.2...,RCRGRSAREVRENSGVVVSTSGVRGKTVELSMGPNAISNKTYYFDK...,PF00225,p_loop_gtpase,A0A0D2I9H7_9EURO
1407317,1407317,A0A428QG75_9HYPO/43-437,A0A428QG75_9HYPO/43-437 A0A428QG75.1 PF00225.2...,ASDEDSRTAVKVAVRVRPPLKSTDPGYDLIPQRFQRSMVQTTSDTS...,PF00225,p_loop_gtpase,A0A428QG75_9HYPO
1407318,1407318,A0A2R6QVF3_ACTCH/9-326,A0A2R6QVF3_ACTCH/9-326 A0A2R6QVF3.1 PF00225.24...,RARPLSPEDAKTSPWRISGNSIFIPNHSSKFEFDQIFGENCKTREV...,PF00225,p_loop_gtpase,A0A2R6QVF3_ACTCH
1407319,1407319,A0A287IHV8_HORVV/1-229,A0A287IHV8_HORVV/1-229 A0A287IHV8.1 PF00225.24...,MHGEQKSPGIIPLAVKDVFSIIQDTPGREFLLRVSYLEIYNEVIND...,PF00225,p_loop_gtpase,A0A287IHV8_HORVV


In [44]:
def get_uniprot_entry(acc):
    url = "https://www.uniprot.org/uniprot/?query="+acc+"&sort=score&columns=id,entry name,reviewed,protein names,genes,organism,organism-id,lineage(all),sequence,length&format=tab"
    try:
        urlData = requests.get(url).content
        rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')),sep = '\t')
    except:
        rawData = pd.DataFrame()
    return rawData 

In [45]:
a = get_uniprot_entry("A0A428QG75_9HYPO")
b = get_uniprot_entry("A0A2R6QVF3_ACTCH")

In [48]:
kinesin_uniprot = get_uniprot_entry(kinesin["uniprot_acc"].iloc[0])
for i in range(1,kinesin.shape[0]):
    curr = get_uniprot_entry(kinesin["uniprot_acc"].iloc[i])
    if not (curr.empty):
        kinesin_uniprot = pd.concat([kinesin_uniprot,curr], ignore_index=True)
    if i%500 ==0:
        print(kinesin_uniprot.shape)

(501, 10)
(1003, 10)
(1504, 10)
(2005, 10)
(2505, 10)
(3005, 10)
(3509, 10)
(4009, 10)
(4511, 10)
(5012, 10)
(5513, 10)
(6013, 10)
(6513, 10)
(7013, 10)
(7513, 10)
(8014, 10)
(8514, 10)
(9014, 10)
(9517, 10)
(10018, 10)
(10518, 10)
(11019, 10)
(11519, 10)
(12020, 10)
(12521, 10)
(13023, 10)
(13523, 10)
(14025, 10)
(14525, 10)
(15025, 10)


In [49]:
tubulin_uniprot = get_uniprot_entry(tubulin["uniprot_acc"].iloc[0])
for i in range(1,tubulin.shape[0]):
    curr = get_uniprot_entry(kinesin["uniprot_acc"].iloc[i])
    if not (curr.empty):
        tubulin_uniprot = pd.concat([tubulin_uniprot,curr], ignore_index=True)
    if i%500 ==0:
        print(tubulin_uniprot.shape)

(501, 10)
(1003, 10)
(2005, 10)
(2505, 10)
(3005, 10)
(3509, 10)
(4009, 10)
(4510, 10)
(5011, 10)
(5512, 10)


In [50]:
print('done')

done


In [52]:
tubulin_uniprot.shape

(5962, 10)

In [53]:
kinesin_uniprot.shape

(15123, 10)

In [54]:
tubulin_uniprot.to_csv("../../data/david_data/tubulin.csv",index = False)
kinesin_uniprot.to_csv("../../data/david_data/kinesin.csv",index = False)

In [55]:
print('done')

done


## Inspect whether the organisms of top interests are included in the dataset

- Giardia lamblia: Giardia intestinalis
- Paramecium: Paramecium
- Diatom: Bacillariophyta
- Dileptus sp.: Dileptus
- Enchelyodon sp.: Enchelyodon
- Opisthokonta: Opisthokonta
- Amoebozoa: Amoebozoa
- Hemimastigophora: Hemimastigophora 
- Euglenozoa: Euglenozoa 
- Archaeplastida: Rhodophyta,Rhodelphida,Glaucocystophyceae,Viridiplantae,Picozoa,Epimorpha
- Carpediemonas

In [3]:
tubulin_uniprot = pd.read_csv("../../data/david_data/tubulin.csv")

,Entry,Entry name,Status,Protein names,Gene names,Organism,Organism ID,Taxonomic lineage (all),Sequence,Length
0,V9HMW9,V9HMW9_9NEIS,unreviewed,Cell division protein FtsZ,ftsZ HMPREF9021_00284,Simonsiella muelleri ATCC 29453,641147.0,"cellular organisms, Bacteria, Proteobacteria, ...",MDKPIYNFVEAPTGPAVIKVIGVGGGGCNAINNMIENPIQGVEYIS...,418.0
1,A0A0D2I9H7,A0A0D2I9H7_9EURO,unreviewed,Kinesin motor domain-containing protein,Z520_10451,Fonsecaea multimorphosa CBS 102226,1442371.0,"cellular organisms, Eukaryota, Opisthokonta, F...",MSGSQRHANTATRTSMRPPPRPHVPQYPSITGRAPSPAESLSSSGA...,1209.0
2,A0A428QG75,A0A428QG75_9HYPO,unreviewed,Kinesin motor domain-containing protein,CEP54_004803,Fusarium sp. AF-8,1325734.0,"cellular organisms, Eukaryota, Opisthokonta, F...",MASSPPTSPGGIQRPLSAAISRAAPRSTSRLSMTSKTGGGSRASDE...,1725.0
3,A0A2R6QVF3,A0A2R6QVF3_ACTCC,unreviewed,Kinesin-like protein,CEY00_Acc13213,Actinidia chinensis var. chinensis (Chinese so...,1590841.0,"cellular organisms, Eukaryota, Viridiplantae, ...",MERIHVTVRARPLSPEDAKTSPWRISGNSIFIPNHSSKFEFDQIFG...,1325.0
4,A0A287IHV8,A0A287IHV8_HORVV,unreviewed,Kinesin motor domain-containing protein,NaN,Hordeum vulgare subsp. vulgare (Domesticated b...,112509.0,"cellular organisms, Eukaryota, Viridiplantae, ...",MHGEQKSPGIIPLAVKDVFSIIQDTPGREFLLRVSYLEIYNEVIND...,690.0


In [4]:
tubulin_uniprot["Taxonomic lineage (all)"].head()

0    cellular organisms, Bacteria, Proteobacteria, ...
1    cellular organisms, Eukaryota, Opisthokonta, F...
2    cellular organisms, Eukaryota, Opisthokonta, F...
3    cellular organisms, Eukaryota, Viridiplantae, ...
4    cellular organisms, Eukaryota, Viridiplantae, ...
Name: Taxonomic lineage (all), dtype: object

In [5]:
organism_ls = ["Giardia intestinalis","Paramecium","Bacillariophyta","Dileptus","Enchelyodon","Opisthokonta",\
               "Amoebozoa","Hemimastigophora","Euglenozoa","Rhodophyta","Rhodelphida","Glaucocystophyceae","Viridiplantae",\
               "Picozoa","Epimorpha"]              
               

In [8]:
lowered_description = tubulin_uniprot["Taxonomic lineage (all)"].str.lower()
tubulin_uniprot["og"] = "unlabeled"

for label in organism_ls:
    print(label)
    label = label.lower() 
    boo = lowered_description.str.contains( label,na=False)
    tubulin_uniprot.loc[boo,"og"] = label

Giardia intestinalis
Paramecium
Bacillariophyta
Dileptus
Enchelyodon
Opisthokonta
Amoebozoa
Hemimastigophora
Euglenozoa
Rhodophyta
Rhodelphida
Glaucocystophyceae
Viridiplantae
Picozoa
Epimorpha


In [16]:
tubulin_uniprot.groupby("og").count().iloc[:,0:1]

,Entry
og,
amoebozoa,35
bacillariophyta,12
euglenozoa,104
giardia intestinalis,7
opisthokonta,3597
paramecium,37
rhodophyta,15
unlabeled,568
viridiplantae,1587


In [10]:
kinesin_uniprot = pd.read_csv("../../data/david_data/kinesin.csv")

In [11]:
lowered_description = kinesin_uniprot["Taxonomic lineage (all)"].str.lower()
kinesin_uniprot["og"] = "unlabeled"

for label in organism_ls:
    print(label)
    label = label.lower() 
    boo = lowered_description.str.contains( label,na=False)
    kinesin_uniprot.loc[boo,"og"] = label

Giardia intestinalis
Paramecium
Bacillariophyta
Dileptus
Enchelyodon
Opisthokonta
Amoebozoa
Hemimastigophora
Euglenozoa
Rhodophyta
Rhodelphida
Glaucocystophyceae
Viridiplantae
Picozoa
Epimorpha


In [15]:
kinesin_uniprot.groupby("og").count().iloc[:,0:1]

,Entry
og,
amoebozoa,121
bacillariophyta,42
euglenozoa,276
giardia intestinalis,30
opisthokonta,9198
paramecium,97
rhodophyta,34
unlabeled,1569
viridiplantae,3756


In [13]:
kinesin_uniprot.head()

,Entry,Entry name,Status,Protein names,Gene names,Organism,Organism ID,Taxonomic lineage (all),Sequence,Length,og
0,V2XA84,V2XA84_MONRO,unreviewed,Kinesin-like protein,Moror_16175,Moniliophthora roreri (strain MCA 2997) (Cocoa...,1381753.0,"cellular organisms, Eukaryota, Opisthokonta, F...",MASTRRIRIATRLRPRIDGEIDDGAIQVCHASDNTAASSSSASSTT...,742.0,opisthokonta
1,A0A0D2I9H7,A0A0D2I9H7_9EURO,unreviewed,Kinesin motor domain-containing protein,Z520_10451,Fonsecaea multimorphosa CBS 102226,1442371.0,"cellular organisms, Eukaryota, Opisthokonta, F...",MSGSQRHANTATRTSMRPPPRPHVPQYPSITGRAPSPAESLSSSGA...,1209.0,opisthokonta
2,A0A428QG75,A0A428QG75_9HYPO,unreviewed,Kinesin motor domain-containing protein,CEP54_004803,Fusarium sp. AF-8,1325734.0,"cellular organisms, Eukaryota, Opisthokonta, F...",MASSPPTSPGGIQRPLSAAISRAAPRSTSRLSMTSKTGGGSRASDE...,1725.0,opisthokonta
3,A0A2R6QVF3,A0A2R6QVF3_ACTCC,unreviewed,Kinesin-like protein,CEY00_Acc13213,Actinidia chinensis var. chinensis (Chinese so...,1590841.0,"cellular organisms, Eukaryota, Viridiplantae, ...",MERIHVTVRARPLSPEDAKTSPWRISGNSIFIPNHSSKFEFDQIFG...,1325.0,viridiplantae
4,A0A287IHV8,A0A287IHV8_HORVV,unreviewed,Kinesin motor domain-containing protein,NaN,Hordeum vulgare subsp. vulgare (Domesticated b...,112509.0,"cellular organisms, Eukaryota, Viridiplantae, ...",MHGEQKSPGIIPLAVKDVFSIIQDTPGREFLLRVSYLEIYNEVIND...,690.0,viridiplantae


In [14]:
tubulin_uniprot.to_csv("../../data/david_data/tubulin.csv",index = False)
kinesin_uniprot.to_csv("../../data/david_data/kinesin.csv",index = False)